<a href="https://colab.research.google.com/github/tbowetum/sparkProject/blob/main/02_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium
!pip install pyspark

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 59.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=628aa2cd00f1a896ea763f7f95a010814cc116ba229ed85e7b6674258bc9d9f6
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import os
import folium
import pandas as pd
import numpy as np
from folium import plugins
import branca.colormap as cm
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
from pyspark.sql import SparkSession



SPARK SESSION

In [ ]:
spark = SparkSession\
        .builder\
        .appName("projetSpark")\
        .getOrCreate()

# Obtain data

In [ ]:
velibs =  spark.read.csv('C:/Users/bowet/Documents/pythonProject/velib.csv', inferSchema=True, header=True)
velibs.show(10)
velibs.registerTempTable("velib_paris")

Py4JJavaError: ignored

In [ ]:
output = velibs.select('commune')
output.show(5)

In [ ]:
velibs.groupby("commune").count().show()

In [ ]:
velibs = velibs[velibs.Station_en_fonctionnement == 'OUI']

In [ ]:
velibs.show()

In [ ]:
#Selection and summary statistics of all numeric variables
num_cols = [col[0] for col in velibs.dtypes if col[1] != 'string']
velibs.select(num_cols).describe().show() 

In [ ]:
# mean of numerical variables relative to the target variable
for col in [col.lower() for col in num_cols]:
    velibs.groupby("commune").agg({col: "mean"}).show()

# Number of bike stands

In [ ]:
bike_stands_arron = velibs.groupby('commune').sum('Nombre_bornettes_libres')
pds=bike_stands_arron.toPandas()

pds.set_index('commune', inplace=True)

pds.head()


In [ ]:
pds.plot(kind='line',figsize=(12,6))

# Available bikes

In [ ]:
velo_dispo = velibs.groupby('Nom_station').sum('Nombretotal_velos_disponibles')
pds=velo_dispo.toPandas()


pds.set_index('Nom_station', inplace=True)

pds.head()

In [ ]:
# pds.plot(kind="bar", figsize = (30, 10))

# Vélo mécanique disponible

In [ ]:
Velos_mecaniques_disponibles = velibs.sql('SELECT Nom_station, Velos_mecaniques_disponibles from velib_paris where commune = "Paris"')
Velos_mecaniques_disponibles.show()


# Vélo électrique disponible

In [ ]:
Velos_electriques_disponibles = velibs.sql('SELECT Nom_station, Velos_electriques_disponibles from velib_paris where commune = "Paris" ')
Velos_electriques_disponibles.show()


In [ ]:
<div style="display:fill;
            border-radius: false;
            border-style: solid;
            border-color:#000000;
            border-style: false;
            border-width: 2px;
            color:#CF673A;
            font-size:15px;
            font-family: Georgia;
            background-color:#E8DCCC;
            text-align:center;
            letter-spacing:0.1px;
            padding: 0.1em;">
**<h2>♡ QUESTION? ♡**

Ref

Dataset

https://opendata.paris.fr/explore/?disjunctive.theme&disjunctive.publisher&disjunctive.keyword&disjunctive.modified&disjunctive.features&sort=modified